# Feature Engineering

In [14]:
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [15]:
import nbimporter
from pre_processing import load_train

### Funciones auxiliares

In [16]:
def corr(df):
    return df.corr()['precio'].sort_values(ascending=False)[1:]

In [17]:
def aniomes(anio, mes):
    if len(str(mes)) == 1:
        return int(str(anio)+'0'+str(mes))
    return int(str(anio)+str(mes))

In [18]:
def contiene_indicador_jardin(descripcion):
    indicadores = ["jardin","patio"]
    for indicador in indicadores:
        if indicador in str(descripcion).lower():
            return 1
    return 0

In [19]:
def contiene_vigilancia(descripcion):
    indicadores = ["vigilancia", "seguridad"]
    for indicador in indicadores:
        if indicador in str(descripcion).lower():
            return 1
    return 0

# Desarollo

In [20]:
df = load_train()

In [21]:
df.head(2)

,titulo,descripcion,direccion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Apartamento,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,2015-08-23,0,0,0,0,0,2273000.0
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,Casa en condominio,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,2013-06-28,0,0,0,1,1,3600000.0


In [22]:
features = {}
features['originales'] = df.columns.values

### Variables categoricas. Encoding.

In [23]:
var_categoricas = ['tipodepropiedad', 'ciudad', 'provincia']

In [24]:
# label encoding
encoder = LabelEncoder()
encoded = df[var_categoricas].apply(encoder.fit_transform).add_suffix("_labelencoded")
encoded.to_csv('./features/categoricas_labelencoded.csv')
features['categoricas_labelencoded'] = encoded.columns.values

In [25]:
# onehot 

# para tipodepropiedad
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['tipodepropiedad']).add_suffix("_ohencoded")
encoded.to_csv('./features/tipodepropiedad_ohencoded.csv')
features['tipodepropiedad_ohencoded'] = encoded.columns.values

# para provincias
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['provincia']).add_suffix("_ohencoded")
encoded.to_csv('./features/provincia_ohencoded.csv')
features['provincia_ohencoded'] = encoded.columns.values

# para ciudades
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['ciudad']).add_suffix("_ohencoded")
encoded.to_csv('./features/ciudad_ohencoded.csv')
features['ciudad_ohencoded'] = encoded.columns.values

### Generacion de nuevas features

In [26]:
df.columns

Index(['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

##### FECHA

In [27]:
df_fecha = df[['fecha', 'precio', 'metroscubiertos']].copy()

df_fecha['anio'] = df_fecha['fecha'].dt.year
df_fecha['mes'] = df_fecha['fecha'].dt.month
df_fecha['dia'] = df_fecha['fecha'].dt.day
df_fecha['timestamp'] = df_fecha['fecha'].apply(lambda x: datetime.timestamp(x))
df_fecha['aniomes'] = df_fecha.apply(lambda row: aniomes(row['anio'], row['mes']), axis=1)

In [28]:
# calculamos el precio promedio del metro cubierto por aniomes
precio_mtcubierto_aniomes = df_fecha.groupby(['aniomes'])['metroscubiertos', 'precio'].agg(sum).apply(lambda x: x['precio']/x['metroscubiertos'], axis=1)
df_fecha['precio_promedio_metrocubierto_aniomes'] = df_fecha['aniomes'].apply(lambda x: precio_mtcubierto_aniomes[x])

In [63]:
# calculamos el precio promedio del metro cubierto por mes
precio_mtcubierto_mes = df_fecha.groupby(['mes'])['metroscubiertos', 'precio'].agg(sum).apply(lambda x: x['precio']/x['metroscubiertos'], axis=1)
df_fecha['precio_promedio_metrocubierto_mes'] = df_fecha['mes'].apply(lambda x: precio_mtcubierto_mes[x])

In [64]:
df_fecha = df_fecha.drop(['precio', 'metroscubiertos'], axis=1)
pasar_a_enteros = ['precio_promedio_metrocubierto_mes', 'precio_promedio_metrocubierto_aniomes']
df_fecha[pasar_a_enteros] = df_fecha[pasar_a_enteros].astype('uint16')

In [65]:
df_fecha.to_csv('./features/fecha.csv')
features['fecha'] = df_fecha.columns.values

In [66]:
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_fecha['aniomes'].astype('category')).add_suffix("_ohencoded")
encoded.to_csv('./features/aniomes_ohencoded.csv')
features['aniomes_ohencoded'] = encoded.columns.values

In [67]:
df_fecha.head()

,fecha,anio,mes,dia,timestamp,aniomes,precio_promedio_metrocubierto_aniomes,precio_promedio_metrocubierto_mes
id,,,,,,,,
254099,2015-08-23,2015,8,23,1.440299e+09,201508,16183,15363
53461,2013-06-28,2013,6,28,1.372388e+09,201306,14297,15521
247984,2015-10-17,2015,10,17,1.445051e+09,201510,15996,15547
209067,2012-03-09,2012,3,9,1.331262e+09,201203,13039,15423
185997,2016-06-07,2016,6,7,1.465268e+09,201606,16142,15521


##### DESCRIPCION

In [29]:
df.columns

Index(['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [30]:
df_descripcion = df[['descripcion']].copy()

df_descripcion["jardin"] = df_descripcion["descripcion"].map(contiene_indicador_jardin)
df_descripcion["vigilancia"] = df_descripcion["descripcion"].map(contiene_vigilancia)

In [31]:
df_descripcion.head(2)

,descripcion,jardin,vigilancia
id,,,
254099,"depto. interior de 80.15m2, consta de sala com...",0,0
53461,"<p>entre sonora y guerrero, atr&aacute;s del h...",0,0


In [32]:
df_descripcion.drop('descripcion', inplace=True, axis=1)
df_descripcion.to_csv('./features/descripcion.csv')
features['descripcion'] = df_descripcion.columns.values

Promedio y desvio estandar de superficie cubierta en la ciudad

In [52]:
df_superficie_cubierta = df[['metroscubiertos','ciudad']].copy().reset_index()

In [53]:
avg_metroscubiertos = df_superficie_cubierta.groupby('ciudad').agg({"metroscubiertos":["mean","std"]}).reset_index()
avg_metroscubiertos.columns = ['ciudad','metroscubiertos_mean','metroscubiertos_std']

In [54]:
df_superficie_cubierta_mean_std = pd.merge(df_superficie_cubierta,avg_metroscubiertos,how='inner',on='ciudad').set_index('id')

In [55]:
df_superficie_cubierta_mean_std = df_superficie_cubierta_mean_std[["metroscubiertos_mean","metroscubiertos_std"]]

In [56]:
df_superficie_cubierta_mean_std.to_csv('./features/superficie_mean_std.csv')

Promedio y desvio estandar de antiguedad cubierta en la ciudad

In [45]:
df_antiguedad = df[['antiguedad','ciudad']].copy().reset_index()

In [46]:
avg_antiguedad = df_antiguedad.groupby('ciudad').agg({"antiguedad":["mean","std"]}).reset_index()
avg_antiguedad.columns = ['ciudad','antiguedad_mean','antiguedad_std']

In [47]:
df_antiguedad_mean_std = pd.merge(df_antiguedad,avg_antiguedad,how='inner',on='ciudad').set_index('id')

In [49]:
df_antiguedad_mean_std

,antiguedad,ciudad,antiguedad_mean,antiguedad_std
id,,,,
254099,NaN,Benito Juárez,10.569339,12.252472
130216,20.0,Benito Juárez,10.569339,12.252472
73348,5.0,Benito Juárez,10.569339,12.252472
244353,0.0,Benito Juárez,10.569339,12.252472
237577,6.0,Benito Juárez,10.569339,12.252472
...,...,...,...,...
79273,5.0,Bahía de Banderas,3.412826,3.119107
204615,5.0,Bahía de Banderas,3.412826,3.119107
288543,0.0,Bahía de Banderas,3.412826,3.119107


In [58]:
df_antiguedad_mean_std = df_antiguedad_mean_std[["antiguedad_mean","antiguedad_std"]]
df_antiguedad_mean_std.to_csv('./features/antiguedad_mean_std.csv')

In [59]:
def load_engineered_dataset():
    df = load_train()
    df = df[["centroscomercialescercanos","escuelascercanas","metroscubiertos","metrostotales","banos","antiguedad","habitaciones","garages","piscina","gimnasio","precio"]]
    
    #Agregado de anio-mes en one hot encoding
    df_aniomes_ohencoded = pd.read_csv("features/aniomes_ohencoded.csv", index_col='id')
    df = pd.merge(df,df_aniomes_ohencoded,how='left',left_index=True, right_index=True)
    
    #Agregado de provincia en one hot encoding
    df_provincia_ohencoded = pd.read_csv("features/provincia_ohencoded.csv", index_col='id')
    df = pd.merge(df,df_provincia_ohencoded,how='left',left_index=True, right_index=True)
    
    #Agregado de tipodepropiedad en one hot encoding
    df_tipo_ohencoded = pd.read_csv("features/tipodepropiedad_ohencoded.csv", index_col='id')
    df = pd.merge(df,df_tipo_ohencoded,how='left',left_index=True, right_index=True)
    
    #Agregado de ciudad en one hot encoding
    df_ciudad_ohencoded = pd.read_csv("features/ciudad_ohencoded.csv", index_col='id')
    df = pd.merge(df,df_ciudad_ohencoded,how='left',left_index=True, right_index=True)
    
    
    #Agregado de features obtenidos de la descripcion
    df_descripcion = pd.read_csv("features/descripcion.csv", index_col='id')
    df = pd.merge(df,df_descripcion,how='left',left_index=True, right_index=True)
    
        
    #Agregado de superficie mean y std por ciudad
    df_datos_superficie = pd.read_csv("features/superficie_mean_std.csv", index_col='id')
    df = pd.merge(df,df_datos_superficie,how='left',left_index=True, right_index=True)
    
    #Agregado de antiguedad mean y std por ciudad
    df_datos_antiguedad = pd.read_csv("features/antiguedad_mean_std.csv", index_col='id')
    df = pd.merge(df,df_datos_antiguedad,how='left',left_index=True, right_index=True)
    
    df = df.reset_index()
    df.pop('id')
    
    return df